In [ ]:
# 1️⃣ Install dependencies
!pip install flask pyngrok transformers accelerate bitsandbytes torchvision datasets evaluate pillow opencv-python
!mkdir -p templates static uploads

In [ ]:
# 2️⃣ Hugging Face login
import os
from huggingface_hub import login, whoami

os.environ["HF_TOKEN"] = "your_huggingface_token_here"  # Replace with your actual token

token = os.environ.get("HF_TOKEN")
if not token:
    raise SystemExit("HF_TOKEN not set.")

login(token=token)
try:
    info = whoami()
    user = info.get("name") or info.get("login") or "unknown"
    print(f"Logged in as: {user}")
except Exception as e:
    print(f"Login verification failed: {e}")

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive/Colab Notebooks/On-Going Projects/Deep Fake Detection /Copy of best_model.pth"

'/content/drive/My Drive/Colab Notebooks/On-Going Projects/Deep Fake Detection /Copy of best_model.pth'


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify
import functools, os, torch, numpy as np, base64, cv2, uuid, re
from PIL import Image
from io import BytesIO
from torchvision import models, transforms
from torch import nn
from collections import deque

app = Flask(__name__)
app.secret_key = "deepfake_app_" + str(uuid.uuid4())[:8]
CKPT_PATH = "/content/drive/My Drive/Colab Notebooks/On-Going Projects/Deep Fake Detection /Copy of best_model.pth"

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
recent_preds = deque(maxlen=5)

def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Build model exactly as in your working Kaggle script ---
@functools.lru_cache(maxsize=1)
def get_model():
    device = get_device()
    print(f"📦 Loading model from: {CKPT_PATH}")
    state = torch.load(CKPT_PATH, map_location="cpu")

    # Detect FC structure dynamically
    keys = list(state.keys())
    pattern = re.compile(r"^fc\.(\d+)\.")
    fc_indices = sorted({int(pattern.match(k).group(1)) for k in keys if pattern.match(k)})

    modules = []
    for i in range(max(fc_indices) + 1):
        prefix = f"fc.{i}."
        w_key = prefix + "weight"
        running_mean_key = prefix + "running_mean"
        if w_key in state and state[w_key].ndim == 2:  # Linear
            out_f, in_f = state[w_key].shape
            modules.append(nn.Linear(in_f, out_f))
        elif w_key in state and state[w_key].ndim == 1 and running_mean_key in state:
            num_feat = state[w_key].shape[0]
            modules.append(nn.BatchNorm1d(num_feat))
        else:
            modules.append(nn.Identity())

    model = models.resnet50(weights=None)
    model.fc = nn.Sequential(*modules)

    model_state = model.state_dict()
    filtered = {k: v for k, v in state.items() if k in model_state and v.shape == model_state[k].shape}
    model_state.update(filtered)
    model.load_state_dict(model_state, strict=False)

    model.to(device)
    model.eval()
    print("✅ Model loaded successfully and ready!")
    return model, device

# Preprocessing (same as Kaggle)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def prepare_image(image_bytes):
    img = Image.open(BytesIO(image_bytes)).convert("RGB")
    return preprocess(img).unsqueeze(0)

def infer(image_bytes):
    model, device = get_model()
    x = prepare_image(image_bytes).to(device)
    with torch.no_grad():
        logits = model(x)
        probs = torch.nn.functional.softmax(logits, dim=1)[0].cpu().numpy()

    fake_prob, real_prob = float(probs[0]), float(probs[1])
    pred = "REAL" if real_prob > fake_prob else "AI-GENERATED"

    return {"prediction": pred, "fake_probability": fake_prob, "real_probability": real_prob}

@app.route("/")
def home():
    return render_template("home.html")

@app.route("/photo")
def photo_page():
    return render_template("index.html")

@app.route("/webcam")
def webcam_page():
    return render_template("webcam.html")

@app.route("/predict", methods=["POST"])
def predict():
    try:
        file = request.files.get("image")
        if not file:
            return jsonify({"error": "No image provided"}), 400
        img_bytes = file.read()
        return jsonify(infer(img_bytes))
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/analyze_frame", methods=["POST"])
def analyze_frame():
    try:
        data = request.get_json()
        if not data or "frame" not in data:
            return jsonify({"error": "No frame data"}), 400

        frame_data = data["frame"].split(",")[1]
        frame_bytes = base64.b64decode(frame_data)
        np_arr = np.frombuffer(frame_bytes, np.uint8)
        frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(80, 80))
        if len(faces) > 0:
            x, y, w, h = max(faces, key=lambda b: b[2] * b[3])
            margin = int(0.25 * h)
            y1, y2 = max(0, y - margin), min(frame.shape[0], y + h + margin)
            x1, x2 = max(0, x - margin), min(frame.shape[1], x + w + margin)
            face_crop = frame[y1:y2, x1:x2]
        else:
            face_crop = frame

        # Convert to PIL for preprocess
        face_pil = Image.fromarray(face_crop)
        x = preprocess(face_pil).unsqueeze(0).to(get_device())

        # Inference
        model, device = get_model()
        with torch.no_grad():
            logits = model(x)
            probs = torch.nn.functional.softmax(logits, dim=1)[0].cpu().numpy()

        fake_prob, real_prob = float(probs[0]), float(probs[1])
        recent_preds.append(real_prob)
        avg_real = np.mean(recent_preds)
        avg_fake = 1 - avg_real

        if avg_real > 0.6:
            pred = "REAL"
        elif avg_fake > 0.6:
            pred = "AI-GENERATED"
        else:
            pred = "UNCERTAIN"

        return jsonify({
            "prediction": pred,
            "fake_probability": float(avg_fake),
            "real_probability": float(avg_real)
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    print(f"🚀 Deepfake Detector running on {get_device()}")
    app.run(host="0.0.0.0", port=8000, debug=False)


Overwriting app.py


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>TruthLens • Deepfake Detector</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  <style>
    .container {
      display: flex;
      justify-content: center;
      align-items: flex-start;
      flex-direction: column;
      gap: 20px;
      max-width: 800px;
      margin: 40px auto;
      padding: 20px;
      background: rgba(255,255,255,0.05);
      border-radius: 16px;
      box-shadow: 0 8px 25px rgba(0,0,0,0.4);
    }
    .preview {
      width: 300px;
      border-radius: 12px;
      box-shadow: 0 0 10px rgba(0,0,0,0.3);
      margin-top: 10px;
    }
    .center { text-align: center; width: 100%; }
    h2 { color: #00d4ff; }
  </style>
</head>
<body>
  <header class="nav">
    <div class="brand">🛰️ <strong>TruthLens</strong></div>
    <div class="nav-right"><a href="/" style="color:#00d4ff;text-decoration:none;">🏠 Home</a></div>
  </header>

  <div class="container">
    <div class="center">
      <h2>📸 Upload an Image</h2>
      <form id="uploadForm" enctype="multipart/form-data">
        <input type="file" name="image" accept="image/*" id="imageInput" required>
        <div>
          <button type="submit" class="btn primary">Analyze</button>
          <button type="button" id="clearBtn" class="btn ghost">Clear</button>
        </div>
        <img id="previewImg" class="preview" src="" alt="Preview" style="display:none;">
      </form>
      <p id="status" class="status">No image uploaded.</p>
    </div>

    <div id="resultCard" class="hidden center">
      <div class="verdict" id="verdict">—</div>
      <p>AI: <span id="fakePct">0%</span> | Real: <span id="realPct">0%</span></p>
      <div id="errorCard" class="error hidden"></div>
    </div>
  </div>

  <footer class="footer">Built with ❤️ — Upload clear facial images for best results</footer>

<script>
const form = document.getElementById("uploadForm");
const imgInput = document.getElementById("imageInput");
const previewImg = document.getElementById("previewImg");
const resultCard = document.getElementById("resultCard");
const verdict = document.getElementById("verdict");
const fakePct = document.getElementById("fakePct");
const realPct = document.getElementById("realPct");
const status = document.getElementById("status");
const errorCard = document.getElementById("errorCard");
const clearBtn = document.getElementById("clearBtn");

imgInput.addEventListener("change", () => {
  const f = imgInput.files[0];
  if (f) {
    previewImg.src = URL.createObjectURL(f);
    previewImg.style.display = "block";
    status.textContent = "Ready to analyze: " + f.name;
  }
});

form.addEventListener("submit", async (e) => {
  e.preventDefault();
  status.textContent = "Analyzing...";
  const fd = new FormData(form);
  try {
    const res = await fetch("/predict", { method: "POST", body: fd });
    const data = await res.json();
    if (data.error) throw new Error(data.message || data.error);
    verdict.textContent = data.prediction;
    fakePct.textContent = (data.fake_probability * 100).toFixed(1) + "%";
    realPct.textContent = (data.real_probability * 100).toFixed(1) + "%";
    verdict.className = data.prediction === "REAL" ? "verdict real" : "verdict fake";
    resultCard.classList.remove("hidden");
    status.textContent = "✅ Analysis Complete!";
  } catch (err) {
    errorCard.textContent = "Error: " + err.message;
    errorCard.classList.remove("hidden");
    status.textContent = "❌ Failed";
  }
});

clearBtn.addEventListener("click", () => {
  imgInput.value = "";
  previewImg.style.display = "none";
  resultCard.classList.add("hidden");
  errorCard.classList.add("hidden");
  status.textContent = "No image uploaded.";
});
</script>
</body>
</html>


Overwriting templates/index.html


In [ ]:
%%writefile templates/home.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>TruthLens • Deepfake Detection Hub</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  <style>
    h1 {
      font-size: 2.4rem;
      color: #00d4ff;
      text-shadow: 0 0 8px rgba(0, 212, 255, 0.5);
    }
    p {
      color: #9aa3b2;
      font-size: 1.1rem;
      margin-top: 10px;
    }
    .buttons {
      margin-top: 30px;
    }
  </style>
</head>
<body>
  <div class="center-container">
    <h1>🛰️ TruthLens</h1>
    <p>Choose how you'd like to test deepfake detection.</p>
    <div class="buttons">
      <a href="/photo" class="btn primary">📸 Test Using Photo</a>
      <a href="/webcam" class="btn accent">🎥 Live Webcam Detection</a>
    </div>
    <footer class="footer">AI-powered Deepfake Detection — Built with ❤️ for research</footer>
  </div>
</body>
</html>


Overwriting templates/home.html


In [ ]:
%%writefile templates/webcam.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>TruthLens • Live Deepfake Detection</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  <style>
    main {
      text-align: center;
      margin-top: 40px;
    }
    video {
      border-radius: 14px;
      border: 2px solid rgba(255, 255, 255, 0.2);
      box-shadow: 0 0 15px rgba(0, 0, 0, 0.5);
      width: 420px;
      height: 320px;
      transition: box-shadow 0.3s ease;
    }
    video.detected {
      box-shadow: 0 0 25px rgba(0, 212, 255, 0.6);
    }
    .verdict {
      margin-top: 16px;
      padding: 12px 20px;
      border-radius: 10px;
      display: inline-block;
      font-weight: 800;
      font-size: 1.1rem;
      animation: fadeIn 0.5s ease;
    }
    .verdict.real {
      background: rgba(0,255,150,0.25);
      color: #7affb6;
      box-shadow: 0 0 10px rgba(0,255,150,0.3);
    }
    .verdict.fake {
      background: rgba(255,80,80,0.25);
      color: #ffb6b6;
      box-shadow: 0 0 10px rgba(255,80,80,0.3);
    }
    .verdict.uncertain {
      background: rgba(255,220,100,0.25);
      color: #ffe98a;
      box-shadow: 0 0 10px rgba(255,220,100,0.3);
    }
  </style>
</head>
<body>
  <header class="nav">
    <div class="brand">🛰️ TruthLens</div>
    <div class="nav-right"><a href="/" style="color:#00d4ff;text-decoration:none;">🏠 Home</a></div>
  </header>

  <main>
    <h2>🎥 Live Webcam Deepfake Detection</h2>
    <video id="webcam" autoplay playsinline></video>
    <div id="verdict" class="verdict" style="margin-top:12px;">Initializing camera...</div>
  </main>

<script>
const video = document.getElementById('webcam');
const verdict = document.getElementById('verdict');

navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream => {
    video.srcObject = stream;
    verdict.textContent = "Camera Ready ✅";
    setInterval(captureAndAnalyze, 2000);
  })
  .catch(err => verdict.textContent = "Camera access denied: " + err);

async function captureAndAnalyze() {
  const canvas = document.createElement('canvas');
  canvas.width = video.videoWidth;
  canvas.height = video.videoHeight;
  const ctx = canvas.getContext('2d');
  ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
  const dataURL = canvas.toDataURL('image/jpeg');

  try {
    const res = await fetch('/analyze_frame', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ frame: dataURL })
    });
    const result = await res.json();
    if (result.error) throw new Error(result.error);

    const pred = result.prediction;
    const ai = (result.fake_probability * 100).toFixed(1);
    const real = (result.real_probability * 100).toFixed(1);
    verdict.textContent = `${pred} | AI: ${ai}% | Real: ${real}%`;

    verdict.className = "verdict";
    if (pred === "REAL") verdict.classList.add("real");
    else if (pred === "AI-GENERATED") verdict.classList.add("fake");
    else verdict.classList.add("uncertain");

    video.classList.toggle("detected", pred !== "UNCERTAIN");

  } catch (e) {
    verdict.textContent = "⚠️ Error: " + e.message;
    verdict.className = "verdict uncertain";
  }
}
</script>
</body>
</html>


Overwriting templates/webcam.html


In [ ]:
%%writefile static/style.css
:root {
  --bg: #0b1020;
  --card: rgba(255,255,255,0.05);
  --accent: #7c5cff;
  --accent2: #00d4ff;
  --muted: #9aa3b2;
  --white: #eaf0ff;
}
body {
  margin: 0;
  font-family: Inter, system-ui, "Segoe UI";
  background: linear-gradient(180deg, #071025 0%, #0b1020 60%);
  color: var(--white);
}
.nav {
  display: flex;
  justify-content: space-between;
  align-items: center;
  padding: 14px 20px;
  background: rgba(255,255,255,0.05);
  border-radius: 12px;
  margin: 16px;
  box-shadow: 0 6px 20px rgba(0,0,0,0.4);
}
.brand {
  font-weight: bold;
  color: #00d4ff;
  font-size: 1.3rem;
}
.center-container {
  text-align: center;
  margin-top: 15%;
}
.btn {
  display: inline-block;
  padding: 14px 26px;
  margin: 10px;
  border-radius: 12px;
  text-decoration: none;
  font-weight: 700;
  transition: 0.3s;
}
.primary { background: linear-gradient(90deg,var(--accent),var(--accent2)); color: white; }
.accent { background: linear-gradient(90deg,#00ffcc,#0077ff); color: white; }
.btn:hover { transform: scale(1.06); box-shadow: 0 0 12px rgba(0, 212, 255, 0.4); }
.footer {
  text-align: center;
  color: var(--muted);
  font-size: 0.9rem;
  margin-top: 30px;
}
.verdict {
  font-weight: 800;
  font-size: 1.3rem;
  padding: 10px;
  border-radius: 10px;
}
.verdict.real { background: rgba(0,255,150,0.2); color:#7affb6; }
.verdict.fake { background: rgba(255,100,100,0.2); color:#ffb6b6; }
.verdict.uncertain { background: rgba(255,220,100,0.25); color:#ffe98a; }


Overwriting static/style.css


In [ ]:
# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

^C
^C


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 17913

/bin/bash: line 1: kill: (17913) - No such process


In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python app.py > flask.log 2>&1 &

In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = "your_ngrok_auth_token_here"  # 🔑 replace with your token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log


🌍 Public URL: NgrokTunnel: "https://907c2db49f41.ngrok-free.app" -> "http://localhost:8000"
🚀 Deepfake Detector running on cpu
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
Press CTRL+C to quit


In [ ]:
!tail -n 50 flask.log

🚀 Deepfake Detector running on cpu
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
Press CTRL+C to quit
127.0.0.1 - - [10/Nov/2025 07:21:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:21:26] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:21:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Nov/2025 07:22:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:22:04] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:22:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:22:29] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:22:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:23:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:23:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:23:52] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2025 07:24:17